<a href="https://colab.research.google.com/github/Vaibhav21pandit/nn_in_pytorch/blob/main/CNN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader

LeNet Implementation

In [2]:
#Defining the LeNet architecture
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.pool=nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.conv3=nn.Conv2d(in_channels=16,out_channels=120,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.fc1=nn.Linear(in_features=120,out_features=84)
    self.fc2=nn.Linear(in_features=84,out_features=10)

  def forward(self,input_image):
    x=torch.relu(self.conv1(input_image))
    x=self.pool(x)
    x=torch.relu(self.conv2(x))
    x=self.pool(x)
    x=torch.relu(self.conv3(x))
    x=x.reshape(x.shape[0],-1)
    x=torch.relu(self.fc1(x))
    x=self.fc2(x)
    return x

model=LeNet()




In [ ]:
input=torch.rand((1,3,32,32)) #generating noise of size 32x32
print(model(input).shape) #using the generated noise for a forward pass to see if the model output is correct 

torch.Size([1, 10])


VGGNet Implementation \
Paper- https://arxiv.org/abs/1409.1556

In [41]:
#VGG architecture

class VGGNet(nn.Module):
  def __init__(self,input_channels):
    super(VGGNet,self).__init__()
    self.in_channels=input_channels
    self.layers=[64,64,"M",128,128,"M",256,256,256,"M",512,512,512,"M",512,512,512,"M"]
    self.conv_layers=self.conv_layer_creator(self.layers)
    self.fc_seq=nn.Sequential(
         nn.Linear(in_features=(512*7*7),out_features=4096),
         nn.ReLU(),
         nn.Linear(in_features=4096,out_features=4096),
         nn.ReLU(),
         nn.Linear(in_features=4096,out_features=1000)
    )

  
  def forward(self,input_image):
    x=self.conv_layers(input_image)
    print(x.shape)
    print(self.conv_layers)
    # x=nn.Flatten(x)
    x=x.reshape(x.shape[0],-1)
    x=self.fc_seq(x)
    return x

  def conv_layer_creator(self,layers):
    conv_layers=[]
    in_channels=self.in_channels
    for i in layers:
      if i=="M":
        conv_layers.append(nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)))
      else:
        #note the padding factor of 1
        conv_layers.append(nn.Conv2d(in_channels=in_channels,out_channels=i,kernel_size=(3,3),stride=(1,1),padding=(1,1)))
        # A BatchNorm can be added but not implemented in the original paper
        conv_layers.append(nn.ReLU())
        in_channels=i
    return nn.Sequential(*conv_layers)

model=VGGNet(3)




In [42]:
input_image=torch.rand((1,3,224,224))
print(model(input_image).shape)

torch.Size([1, 512, 7, 7])
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU()
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU()
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU()
  (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), p

**Debugging** \
Forgot to add padding in the Conv2d module and was getting error so the following cell shows how to debug a neural net when shapes dont match

In [39]:
#This is how to debug a model
layers=[64,64,"M",128,128,"M",256,256,256,"M",512,512,512,"M",512]
def conv_layer_creator(layers):
  conv_layers=[]
  in_channels=3
  for i in layers:
    if i=="M":
      conv_layers.append(nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)))
    else:
      conv_layers.append(nn.Conv2d(in_channels=in_channels,out_channels=i,kernel_size=(3,3),stride=(1,1)))
      # A BatchNorm can be added but not implemented in the original paper
      conv_layers.append(nn.ReLU())
      in_channels=i
  return nn.Sequential(*conv_layers)
conv_layers=conv_layer_creator(layers)
print(conv_layers(input_image).shape)

torch.Size([1, 512, 6, 6])


Implementing AlexNet \
Paper-https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf

In [27]:
class AlexNet(nn.Module):
  def __init__(self,in_channels):
    super(AlexNet,self).__init__()
    self.in_channels=in_channels
    self.conv1=nn.Conv2d(in_channels=3,out_channels=96,kernel_size=(11,11),stride=(4,4))
    self.pool=nn.MaxPool2d((5,5),stride=(2,2))
    self.conv2=nn.Conv2d(in_channels=96,out_channels=,kernel=(5,5),)
